In [106]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta
import calendar

In [120]:
meter_list = pd.read_csv('meter.csv')
forecast = pd.read_csv('forecast.csv')
rates = pd.read_csv('rate.csv')

# convert NaN to infity otherwise it will give me problems later
rates['aq_max_kwh'] = rates['aq_max_kwh'].fillna(float("inf"))

                     date exit_zone  aq_min_kwh  aq_max_kwh  rate_p_per_kwh
0     2020-04-01 00:00:00       EA1           0     73200.0          0.2652
1     2020-04-01 00:00:00       EA1       73200    732000.0          0.1980
2     2020-04-01 00:00:00       EA1      732000         inf          0.2875
3     2020-04-01 00:00:00       EA2           0     73200.0          0.2970
4     2020-04-01 00:00:00       EA2       73200    732000.0          0.1524
...                   ...       ...         ...         ...             ...
1135  2024-10-01 00:00:00       WM2       73200    732000.0          0.4537
1136  2024-10-01 00:00:00       WM2      732000         inf          0.7534
1137  2024-10-01 00:00:00       WM3           0     73200.0          0.7263
1138  2024-10-01 00:00:00       WM3       73200    732000.0          0.6109
1139  2024-10-01 00:00:00       WM3      732000         inf          0.4928

[1140 rows x 5 columns]


In [132]:
def get_dayrates(row):
    applicable_rates = rates[(rates['exit_zone'] == row['exit_zone']) & 
                        (rates['aq_min_kwh'] <= row['aq_kwh']) & 
                        (row['aq_kwh'] <= rates['aq_max_kwh']) & 
                        (rates['date'] <= row['date'])]
    valid_rates = applicable_rates.loc[applicable_rates['date'].idxmax()]
    return valid_rates['rate_p_per_kwh']




In [133]:
# merge meter list and forecast to get all the relevant data for looking up the rates in one df - and for grouby sum later on
forecast_list = meter_list.merge(forecast, on=['meter_id'])

# initiate the function to retrieve day rates
forecast_list['rate_p_per_kwh'] = forecast_list.apply(get_dayrates, axis=1)

# calculate the daily cost and save into a new column
forecast_list['daily_cost'] = forecast_list['kwh']*forecast_list['rate_p_per_kwh']

      meter_id  aq_kwh exit_zone                 date         kwh  \
0     14676236   28978       EA1  2020-06-01 00:00:00   22.070768   
1     14676236   28978       EA1  2020-06-02 00:00:00   19.170720   
2     14676236   28978       EA1  2020-06-03 00:00:00   23.555111   
3     14676236   28978       EA1  2020-06-04 00:00:00   18.220712   
4     14676236   28978       EA1  2020-06-05 00:00:00   14.196134   
...        ...     ...       ...                  ...         ...   
3407  88357331  484399       SE2  2022-09-27 00:00:00  441.014724   
3408  88357331  484399       SE2  2022-09-28 00:00:00  441.512055   
3409  88357331  484399       SE2  2022-09-29 00:00:00  437.240657   
3410  88357331  484399       SE2  2022-09-30 00:00:00  517.608354   
3411  88357331  484399       SE2  2022-10-01 00:00:00  460.535505   

      rate_p_per_kwh  
0             0.2652  
1             0.2652  
2             0.2652  
3             0.2652  
4             0.2652  
...              ...  
3407      

In [152]:
# drop unessesairy columns
forecast_list = forecast_list.drop(['aq_kwh','exit_zone','date','rate_p_per_kwh'], axis = 1)

# sum values grouped by meter id
summary = forecast_list.groupby('meter_id').sum()

# convert to GBP and apply roundign by 2 decimal points
summary['daily_cost'] = summary['daily_cost']/100
summary = summary.round(2)

# naming convestion as was specified
summary = summary.reset_index()
summary.columns = ['Meter ID', 'Total Estimated Consumption (kWh)', 'Total Cost (£)']

print(summary)

   Meter ID  Total Estimated Consumption (kWh)  Total Cost (£)
0  14676236                            28978.0          100.15
1  34509937                            78324.0          275.49
2  50264822                           265667.0          731.24
3  88357331                           484399.0         1433.16


In [3]:
'''Write a function that generates a list of random meters of any size. Examples of valid 
exit zones can be found in the rate table. You may randomly generate the annual 
quantity'''

import random 
# random

# exit zones all put into a list

# for every row until the defines size is met 
# index number = meter id
# AQ is a random int 
# retrieve random item from list and enter as exit zone

# return completed csv





'Write a function that generates a list of random meters of any size. Examples of valid \nexit zones can be found in the rate table. You may randomly generate the annual \nquantity'

In [4]:
'''Write a function that generates mock consumption data given a list of meters and a 
start date and duration (number of days in the forecast). The data may be completely 
random and it doesn't have to match with the meters' annual quantities either'''


# read meter list
# generate table for each day from start date until datrt date + duration
# generate random floats for each cell

"Write a function that generates mock consumption data given a list of meters and a \nstart date and duration (number of days in the forecast). The data may be completely \nrandom and it doesn't have to match with the meters' annual quantities either"

In [ ]:
'''Write a function that takes as an input a meter list and a consumption forecast table 
and that calculates the transportation cost table (i.e., best take your logic from task 1 
and wrap it in a function). Benchmark this function using meter lists of different sizes 
and consumption forecasts for periods of different lengths. How does the function scale 
for larger sets of data'''

# read in meter list
# read in consumption forecast

# transportation cost calc from Q1

In [ ]:
'''What are your observations after benchmarking? Are there any steps in the cost 
calculation that can be improved? How would you go about improving the performance 
of this calculation'''

